## 01a - Data preparation: Population spine

- **Project:** _Families, households, networks: Rethinking the relational structure of families through large-scale network data_ <br>
- **Authors:** Nicolás Soler (ORCID 0009-0001-4239-9396), Tom Emery, Agnieszka Kanas <br>
- **Last updated:** January 2026 <br>
- **Full research article published in journal:** _Demography_ (2026)

In [ ]:
import yaml
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
# Load YAML configuration
path_config = 'config.yml'
with open(path_config, 'r') as f:
    config = yaml.safe_load(f)

# 1 - Process microdata files

In [ ]:
# Read microdata files and rename columns

# Dictionaries of desired columns and new column names
cols_persoontab = {
    "RINPERSOON":"RINPERSOON",
    "RINPERSOONS":"RINPERSOONS",
    "GBAGESLACHT":"gender_female",
    "GBAGEBOORTEJAAR":"birth_year",
    "GBAGEBOORTEMAAND":"birth_month"
}
cols_overlijdentab = {
    "RINPERSOON":"RINPERSOON",
    "RINPERSOONS":"RINPERSOONS",
    "GBADatumOverlijden":"death_date"
}
cols_adresobjectbus = {
    "RINPERSOON":"RINPERSOON",
    "RINPERSOONS":"RINPERSOONS",
    "SOORTOBJECTNUMMER":"SOORTOBJECTNUMMER",
    "RINOBJECTNUMMER":"RINOBJECTNUMMER",
    "GBADATUMAANVANGADRESHOUDING":"start_date",
    "GBADATUMEINDEADRESHOUDING":"end_date"
}
cols_vslgwbtab = {
    "SOORTOBJECTNUMMER":"SOORTOBJECTNUMMER",
    "RINOBJECTNUMMER":"RINOBJECTNUMMER",
    "bc2020":"buurt_code_2018"
}

# Read files and rename columns
persoontab = pd.read_spss(
    config["data"]["persoontab"],
    usecols=cols_persoontab.keys(),
    convert_categoricals=False
)
persoontab = persoontab.rename(columns=cols_persoontab)

overlijdentab = pd.read_spss(
    config["data"]["overlijdentab"],
    usecols=cols_overlijdentab.keys(),
    convert_categoricals=False
)
overlijdentab = overlijdentab.rename(columns=cols_overlijdentab)

adresobjectbus = pd.read_spss(
    config["data"]["adresobjectbus"],
    usecols=cols_adresobjectbus.keys(),
    convert_categoricals=False
)
adresobjectbus = adresobjectbus.rename(columns=cols_adresobjectbus)

vslgwbtab = pd.read_spss(
    config["data"]["vslgwbtab"],
    usecols=cols_vslgwbtab.keys(),
    convert_categoricals=False
)
vslgwbtab = vslgwbtab.rename(columns=cols_vslgwbtab)

In [ ]:
# Tidy GBAPERSOONTAB

# Subset only individuals with RINPERSOONS=R
persoontab = persoontab[persoontab["RINPERSOONS"]=="R"]

# Drop duplicates if any
persoontab = persoontab.drop_duplicates("RINPERSOON")

# Recode gender
values_gender = {"1":"0", "2":"1"}
persoontab["gender_female"] = persoontab["gender_female"].map(values_gender)

# Fix encoding of missing dates
persoontab["birth_year"] = np.where(persoontab["birth_year"]=="----", np.nan, persoontab["birth_year"])
persoontab["birth_month"] = np.where(persoontab["birth_month"]=="--", np.nan, persoontab["birth_month"])

# Create birth_date column
persoontab["birth_date"] = pd.to_datetime(
    dict(year=persoontab.birth_year, month=persoontab.birth_month, day=1),
    format="%Y%m%d"
)

# Ensure correct data types and NA encoding in columns of interest
persoontab["RINPERSOON"] = persoontab["RINPERSOON"].astype(str)
persoontab["gender_female"] = persoontab["gender_female"].astype("Int64")
persoontab["birth_date"] = persoontab["birth_date"].astype("datetime64[ns]")
persoontab = persoontab.convert_dtypes()

# Select columns of interest
cols_persoontab_final = ["RINPERSOON","gender_female","birth_date"]
persoontab = persoontab[cols_persoontab_final]

In [ ]:
# Tidy GBAOVERLIJDENTAB

# Subset only individuals with RINPERSOONS=R
overlijdentab = overlijdentab[overlijdentab["RINPERSOONS"]=="R"]

# Drop duplicates if any
overlijdentab = overlijdentab.drop_duplicates("RINPERSOON")

# Convert death_date variable to datetime
overlijdentab["death_date"] = pd.to_datetime(
    dict(
        year=overlijdentab.death_date.str.slice(0,4),
        month=overlijdentab.death_date.str.slice(4,6),
        day=overlijdentab.death_date.str.slice(6,8)
    ),
    format="Y%m%d"
)

# Ensure correct data types and NA encoding in columns of interest
overlijdentab["RINPERSOON"] = overlijdentab["RINPERSOON"].astype(str)
overlijdentab["death_date"] = overlijdentab["death_date"].astype("datetime64[ns]")
overlijdentab = overlijdentab.convert_dtypes()

# Select columns of interest
cols_overlijdentab_final = ["RINPERSOON","death_date"]
overlijdentab = overlijdentab[cols_overlijdentab_final]

In [ ]:
# Tidy GBAADRESOBJECTBUS

# Subset only individuals with RINPERSOONS=R
adresobjectbus = adresobjectbus[adresobjectbus["RINPERSOONS"]=="R"]

# Drop duplicates if any
key_adresobjectbus = ["RINPERSOON", "SOORTOBJECTNUMMER", "RINOBJECTNUMMER", "start_date", "end_date"]
adresobjectbus = adresobjectbus.drop_duplicates(key_adresobjectbus)

# Convert start_date and end_date variables to datetime
adresobjectbus["start_date"] = pd.to_datetime(
    dict(
        year=adresobjectbus.start_date.str.slice(0,4),
        month=adresobjectbus.start_date.str.slice(4,6),
        day=adresobjectbus.start_date.str.slice(6,8)
    ),
    format="%Y%m%d"
)

adresobjectbus["end_date"] = pd.to_datetime(
    dict(
        year=adresobjectbus.end_date.str.slice(0,4),
        month=adresobjectbus.end_date.str.slice(4,6),
        day=adresobjectbus.end_date.str.slice(6,8)
    ),
    format="%Y%m%d"
)

# Ensure correct data types and NA encoding in columns of interest
adresobjectbus["RINPERSOON"] = adresobjectbus["RINPERSOON"].astype(str)
adresobjectbus["SOORTOBJECTNUMMER"] = adresobjectbus["SOORTOBJECTNUMMER"].astype(str)
adresobjectbus["RINOBJECTNUMMER"] = adresobjectbus["RINOBJECTNUMMER"].astype(str)
adresobjectbus["start_date"] = adresobjectbus["start_date"].astype("datetime64[ns]")
adresobjectbus["end_date"] = adresobjectbus["end_date"].astype("datetime64[ns]")
adresobjectbus = adresobjectbus.convert_dtypes()

# Select column of interest
cols_adresobjectbus_final = ["RINPERSOON","SOORTOBJECTNUMMER","RINOBJECTNUMMER","start_date","end_date"]
adresobjectbus = adresobjectbus[cols_adresobjectbus_final]

In [ ]:
# Tidy VSLGWBTAB

# Drop duplicates if any
key_vslgwbtab = ["SOORTOBJECTNUMMER","RINOBJECTNUMMER"]
vslgwbtab = vslgwbtab.drop_duplicates(key_vslgwbtab)

# Split buurt code into gemeente, wijk, and buurt codes
cols_vslgwbtab_codes = ["temporary","gemeente","wijk","buurt"]
vslgwbtab[cols_vslgwbtab_codes] = vslgwbtab["buurt_code_2018"].str.split(r"(.{4})(.{2})", expand=True)
vslgwbtab.drop(columns=["temporary"], inplace=True)

# Encode missing gemeente, wijk, and buurt codes properly
vslgwbtab["gemeente"] = np.where(vslgwbtab["gemeente"]=="----", np.nan, vslgwbtab["gemeente"])
vslgwbtab["wijk"] = np.where(vslgwbtab["wijk"]=="--", np.nan, vslgwbtab["wijk"])
vslgwbtab["buurt"] = np.where(vslgwbtab["buurt"]=="--", np.nan, vslgwbtab["buurt"])

# Ensure correct data types and NA encoding in columns of interest
vslgwbtab["SOORTOBJECTNUMMER"] = vslgwbtab["SOORTOBJECTNUMMER"].astype(str)
vslgwbtab["RINOBJECTNUMMER"] = vslgwbtab["RINOBJECTNUMMER"].astype(str)
vslgwbtab["gemeente"] = vslgwbtab["gemeente"].astype(str)
vslgwbtab["wijk"] = vslgwbtab["wijk"].astype(str)
vslgwbtab["buurt"] = vslgwbtab["buurt"].astype(str)
vslgwbtab = vslgwbtab.convert_dtypes()

# Select columns of interest
cols_vslgwbtab_final = ["SOORTOBJECTNUMMER","RINOBJECTNUMMER","gemeente","wijk","buurt"]
vslgwbtab = vslgwbtab[cols_vslgwbtab_final]

# 2 - Create spine

In [ ]:
# 2018 reference date
date_cutoff = datetime.strptime("2018-01-01", "%Y-%m-%d")

# Delete individuals that were over 110 years old as of 01/01/2018
spine = persoontab[persoontab["birth_date"]>"1908-01"]

# Delete individuals that died before 01/01/2018
spine = spine.merge(overlijdentab, how="left", on="RINPERSOON")
spine = spine[~(spine["death_date"]<=date_cutoff)]

# Delete individuals that were not registered in an address as of 01/01/2018

# Identify address per person as of 01/01/2018
addresses = adresobjectbus[
    (adresobjectbus["start_date"]<=date_cutoff) &
    (adresobjectbus["end_date"]>=date_cutoff)
]

# Drop start and end dates
addresses = addresses.drop(columns=["start_date","end_date"])

# Subset those with address as of 01/01/2018 and join address id
spine = spine.merge(addresses, how="inner", on="RINPERSOON")

# Merge gemeente, wijk, and buurt codes
spine = spine.merge(vslgwbtab, how="left", on=["SOORTOBJECTNUMMER","RINOBJECTNUMMER"])

# Calculate age
spine["age"] = date_cutoff.year - spine["birth_date"].dt.year

# Standardise data types
spine = spine.convert_dtypes()

# Store spine
spine.to_csv(config["data"]["spine"], sep=",", lineterminator="\n",
            index=False, header=True,
            encoding="utf-8", compression="infer")